In [ ]:
!pip install neptune-client==0.9.1 --quiet
!pip install pytorch_lightning==1.2.10 --quiet
!pip install transformers --quiet
!pip install datasets --quiet

     |████████████████████████████████| 215kB 5.4MB/s 
     |████████████████████████████████| 829kB 8.6MB/s 
     |████████████████████████████████| 71kB 7.2MB/s 
     |████████████████████████████████| 174kB 17.1MB/s 
     |████████████████████████████████| 133kB 19.6MB/s 
     |████████████████████████████████| 71kB 7.4MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 
     |████████████████████████████████| 849kB 5.4MB/s 
     |████████████████████████████████| 10.6MB 11.2MB/s 
     |████████████████████████████████| 184kB 37.2MB/s 
     |████████████████████████████████| 122kB 41.6MB/s 
     |████████████████████████████████| 276kB 37.5MB/s 
     |████████████████████████████████| 1.3MB 31.7MB/s 
     |████████████████████████████████| 143kB 36.6MB/s 
     |████████████████████████████████| 296kB 37.4MB/s 
ERROR: tensorflow 2.5.0 has requirement tensorboard~=2.5, but you'll have tensorboard 2.4.1 which is incompatible.
     |████████████████████████████████| 2.3MB 5.4MB/s

In [ ]:
import numpy as np
from sklearn.utils import shuffle
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import neptune.new as neptune
import pytorch_lightning
from pytorch_lightning import LightningModule, Trainer

In [ ]:
run = neptune.init(project='d230640/Aula11', api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJmZTg1OGU3Yi1jOWE1LTRjMjEtYTJjNS1hMjMwM2Y3NDRjOTUifQ==')

https://app.neptune.ai/d230640/Aula11/e/AUL7-4


### Importação dos dados

In [ ]:
!wget -nc http://files.fast.ai/data/examples/imdb_sample.tgz
!tar -xzf imdb_sample.tgz

--2021-06-14 13:56:33--  http://files.fast.ai/data/examples/imdb_sample.tgz
Resolving files.fast.ai (files.fast.ai)... 104.26.3.19, 172.67.69.159, 104.26.2.19, ...
Connecting to files.fast.ai (files.fast.ai)|104.26.3.19|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://files.fast.ai/data/examples/imdb_sample.tgz [following]
--2021-06-14 13:56:33--  https://files.fast.ai/data/examples/imdb_sample.tgz
Connecting to files.fast.ai (files.fast.ai)|104.26.3.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 571827 (558K) [application/x-gtar-compressed]
Saving to: ‘imdb_sample.tgz’

imdb_sample.tgz     100%[===================>] 558.42K  2.22MB/s    in 0.2s    

2021-06-14 13:56:34 (2.22 MB/s) - ‘imdb_sample.tgz’ saved [571827/571827]



In [ ]:
import pandas as pd
df = pd.read_csv('imdb_sample/texts.csv')
df.shape
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


In [ ]:
split_rate = 0.8

treino = df[df['is_valid'] == False]
test = df[df['is_valid'] == True]

df = shuffle(treino)

train = treino[0:int(split_rate*treino.shape[0])]
valid = treino[int(split_rate*treino.shape[0]):]

print('treino.shape:', train.shape)
print('test.shape:', valid.shape)
print('test.shape:', test.shape)

treino.shape: (640, 3)
test.shape: (160, 3)
test.shape: (200, 3)


In [ ]:
X_train = train['text']
Y_train = train['label']
X_valid = valid['text']
Y_valid = valid['label']
X_test = test['text']
Y_test = test['label']

print('X_train.head():', X_train.head())
print('Y_train.head():', Y_train.head())

X_train.head(): 0    Un-bleeping-believable! Meg Ryan doesn't even ...
1    This is a extremely well-made film. The acting...
2    Every once in a long while a movie will come a...
3    Name just says it all. I watched this movie wi...
4    This movie succeeds at being one of the most u...
Name: text, dtype: object
Y_train.head(): 0    negative
1    positive
2    negative
3    positive
4    negative
Name: label, dtype: object


In [ ]:
mapeamento = {'positive': True, 'negative': False}
Y_treino_bool = Y_train.map(mapeamento)
Y_valid_bool = Y_valid.map(mapeamento)
Y_test_bool = Y_test.map(mapeamento)

print(Y_treino_bool.head())

0    False
1     True
2    False
3     True
4    False
Name: label, dtype: bool


### Tokenizer disponível pela biblioteca transformers

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
inputs = tokenizer(X_train[0], padding="max_length", truncation=True)

In [ ]:
print(len(inputs['input_ids']),len(inputs['token_type_ids']), len(inputs['attention_mask']))

512 512 512


In [ ]:
from datasets import load_dataset

dataset = load_dataset('csv', data_files='imdb_sample/texts.csv', split='train')
dataset[0]

Using custom data configuration default-b0f0570519de76f2


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-b0f0570519de76f2/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


{'is_valid': False,
 'label': 'negative',
 'text': "Un-bleeping-believable! Meg Ryan doesn't even look her usual pert lovable self in this, which normally makes me forgive her shallow ticky acting schtick. Hard to believe she was the producer on this dog. Plus Kevin Kline: what kind of suicide trip has his career been on? Whoosh... Banzai!!! Finally this was directed by the guy who did Big Chill? Must be a replay of Jonestown - hollywood style. Wooofff!"}

In [ ]:
def label_to_int(v):
    value = {'text': v['text'], 'label': 1 if v['label'] == 'positive' else 0 }
    return value

dataset = dataset.map(label_to_int)
print(dataset[0])


{'is_valid': False, 'label': 0, 'text': "Un-bleeping-believable! Meg Ryan doesn't even look her usual pert lovable self in this, which normally makes me forgive her shallow ticky acting schtick. Hard to believe she was the producer on this dog. Plus Kevin Kline: what kind of suicide trip has his career been on? Whoosh... Banzai!!! Finally this was directed by the guy who did Big Chill? Must be a replay of Jonestown - hollywood style. Wooofff!"}


In [ ]:
train_dataset = dataset.filter(lambda x: x['is_valid'] == False)
test_dataset = dataset.filter(lambda x: x['is_valid'] == True)
train_dataset = train_dataset.remove_columns('is_valid')
test_dataset = test_dataset.remove_columns('is_valid')
print('train_dataset[0]:', train_dataset[0])
print('test_dataset[0]:', test_dataset[0])


train_dataset[0]: {'label': 0, 'text': "Un-bleeping-believable! Meg Ryan doesn't even look her usual pert lovable self in this, which normally makes me forgive her shallow ticky acting schtick. Hard to believe she was the producer on this dog. Plus Kevin Kline: what kind of suicide trip has his career been on? Whoosh... Banzai!!! Finally this was directed by the guy who did Big Chill? Must be a replay of Jonestown - hollywood style. Wooofff!"}
test_dataset[0]: {'label': 1, 'text': "This very funny British comedy shows what might happen if a section of London, in this case Pimlico, were to declare itself independent from the rest of the UK and its laws, taxes & post-war restrictions. Merry mayhem is what would happen.<br /><br />The explosion of a wartime bomb leads to the discovery of ancient documents which show that Pimlico was ceded to the Duchy of Burgundy centuries ago, a small historical footnote long since forgotten. To the new Burgundians, however, this is an unexpected opport

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

train_tokenized_datasets = train_dataset.map(tokenize_function, batched=True)
test_tokenized_datasets = test_dataset.map(tokenize_function, batched=True)

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer",evaluation_strategy = "epoch",gradient_accumulation_steps=4,learning_rate=5e-5,num_train_epochs=6,save_strategy="epoch",fp16_full_eval=False,logging_strategy = "epoch")

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=train_tokenized_datasets, eval_dataset=test_tokenized_datasets,compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.029200,0.790840,0.855000
2,0.020400,0.823602,0.870000
3,0.013900,0.811114,0.865000
4,0.015200,0.719595,0.890000
5,0.001000,0.754913,0.885000
6,0.001700,0.770546,0.880000


TrainOutput(global_step=150, training_loss=0.01354499655465285, metrics={'train_runtime': 1054.255, 'train_samples_per_second': 0.142, 'total_flos': 0, 'epoch': 6.0, 'init_mem_cpu_alloc_delta': 0, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -11816960, 'train_mem_gpu_alloc_delta': 909062144, 'train_mem_cpu_peaked_delta': 11857920, 'train_mem_gpu_peaked_delta': 6503007232})